# 🏗️ Base Model Architecture (29.7% Accuracy)

## Introduction

This notebook documents the implementation and evaluation of the **Base Model**, our initial approach to speech emotion recognition. This model established our benchmark performance of **29.7% accuracy** on the 8-class emotion classification task.

While this performance may seem modest, it represents more than double the random chance accuracy (12.5%) and served as a critical foundation for subsequent architectural improvements.



## Architecture Overview

The Base Model follows a conventional CNN-based architecture for audio processing:

```
                                     Base Model Architecture
┌───────────────┐    ┌───────────────┐    ┌───────────────┐    ┌───────────────┐    ┌───────────────┐
│               │    │               │    │               │    │               │    │               │
│  Audio Input  │───►│  Spectrogram  │───►│  CNN Layers   │───►│  RNN Layers   │───►│  Classifier   │
│               │    │  Extraction   │    │  (Feature     │    │  (Temporal    │    │  (Output      │
│               │    │               │    │   Extraction) │    │   Modeling)   │    │   Layer)      │
└───────────────┘    └───────────────┘    └───────────────┘    └───────────────┘    └───────────────┘
```

### Key Components

1. **Audio Preprocessing**: Conversion of raw audio to mel-spectrograms
2. **Feature Extraction**: Convolutional layers to detect audio patterns
3. **Temporal Modeling**: Simple recurrent layers to capture time dependencies
4. **Classification**: Fully-connected layers with softmax activation

Let's examine each component in detail.



## 1. Audio Preprocessing

Audio preprocessing is a critical step in speech emotion recognition. Raw audio waveforms are converted to mel-spectrograms, which represent the frequency content of the signal over time in a way that approximates human auditory perception.



In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

def extract_features(file_path, sample_rate=22050, n_mels=128, n_fft=2048, hop_length=512):
    """
    Extract mel-spectrogram features from an audio file
    
    Parameters:
        file_path (str): Path to the audio file
        sample_rate (int): Sample rate for loading the audio
        n_mels (int): Number of mel bands
        n_fft (int): FFT window size
        hop_length (int): Hop length for STFT
        
    Returns:
        mel_spectrogram (np.ndarray): Mel-spectrogram features
    """
    # Load audio file
    y, sr = librosa.load(file_path, sr=sample_rate)
    
    # Extract mel-spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(
        y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length
    )
    
    # Convert to decibels
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
    return mel_spectrogram_db

# Example: Display mel-spectrogram for a sample audio file
# file_path = "../samples/neutral_sample.wav"  # Update with actual path
# mel_spec = extract_features(file_path)
#
# plt.figure(figsize=(10, 4))
# librosa.display.specshow(mel_spec, x_axis='time', y_axis='mel', sr=22050, hop_length=512)
# plt.colorbar(format='%+2.0f dB')
# plt.title('Mel-spectrogram')
# plt.tight_layout()
# plt.show()



## 2. Base Model Architecture

Our Base Model uses a combination of convolutional and recurrent layers implemented in PyTorch.



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BaseEmotionModel(nn.Module):
    def __init__(self, num_emotions=8):
        """
        Initialize the base emotion recognition model
        
        Parameters:
            num_emotions (int): Number of emotion classes
        """
        super(BaseEmotionModel, self).__init__()
        
        # Convolutional layers for feature extraction
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Recurrent layer for temporal modeling
        self.gru = nn.GRU(input_size=128 * 16, hidden_size=128, num_layers=2, batch_first=True)
        
        # Fully connected layers for classification
        self.fc1 = nn.Linear(128, 64)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, num_emotions)
        
    def forward(self, x):
        """
        Forward pass through the network
        
        Parameters:
            x (torch.Tensor): Input tensor of shape (batch_size, 1, height, width)
                             representing mel-spectrograms
                             
        Returns:
            torch.Tensor: Logits for each emotion class
        """
        # CNN feature extraction
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        
        # Reshape for GRU
        batch_size, channels, height, width = x.size()
        x = x.permute(0, 3, 1, 2)  # (batch, width, channels, height)
        x = x.reshape(batch_size, width, channels * height)
        
        # RNN temporal modeling
        x, _ = self.gru(x)
        x = x[:, -1, :]  # Take the output from the last time step
        
        # Classification
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Create model instance
base_model = BaseEmotionModel(num_emotions=8)
print(f"Model parameters: {sum(p.numel() for p in base_model.parameters() if p.requires_grad):,}")



## 3. Training Process

The Base Model was trained using the following approach:

- **Optimizer**: Adam with learning rate of 1e-4
- **Loss Function**: Cross-Entropy Loss
- **Batch Size**: 32
- **Epochs**: 50
- **Early Stopping**: Patience of 10 epochs



In [ ]:
def train_base_model(model, train_loader, val_loader, num_epochs=50, learning_rate=1e-4):
    """
    Train the base emotion recognition model
    
    Parameters:
        model (nn.Module): The model to train
        train_loader (DataLoader): DataLoader for training data
        val_loader (DataLoader): DataLoader for validation data
        num_epochs (int): Number of training epochs
        learning_rate (float): Learning rate for optimizer
        
    Returns:
        dict: Training history (losses and accuracies)
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    # Initialize optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    # Early stopping parameters
    best_val_acc = 0
    patience = 10
    patience_counter = 0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Statistics
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += targets.size(0)
            train_correct += predicted.eq(targets).sum().item()
        
        train_loss = train_loss / len(train_loader)
        train_acc = train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                # Statistics
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += targets.size(0)
                val_correct += predicted.eq(targets).sum().item()
        
        val_loss = val_loss / len(val_loader)
        val_acc = val_correct / val_total
        
        # Update history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Print statistics
        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | "
              f"Val Acc: {val_acc:.4f}")
        
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), "base_model_best.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    return history

# Note: Actual training would be executed here with prepared DataLoaders
# history = train_base_model(base_model, train_loader, val_loader)



## 4. Results Analysis

The Base Model achieved an accuracy of **29.7%** on the validation set, which is our benchmark performance. Let's analyze the confusion matrix and classification metrics to understand the model's strengths and weaknesses.



In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def evaluate_model(model, test_loader):
    """
    Evaluate the model on the test set
    
    Parameters:
        model (nn.Module): The trained model
        test_loader (DataLoader): DataLoader for test data
        
    Returns:
        tuple: (all_predictions, all_targets) for further analysis
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    
    return np.array(all_predictions), np.array(all_targets)

# Example: Display results
# class_names = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']
# predictions, targets = evaluate_model(base_model, test_loader)
# 
# # Confusion matrix
# cm = confusion_matrix(targets, predictions)
# cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# 
# plt.figure(figsize=(10, 8))
# sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues',
#             xticklabels=class_names, yticklabels=class_names)
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title('Confusion Matrix - Base Model (29.7%)')
# plt.tight_layout()
# 
# # Classification report
# report = classification_report(targets, predictions, target_names=class_names, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# display(report_df.round(3))



## 5. Base Model Strengths and Limitations

### Strengths

1. **Simplicity**: The model architecture is straightforward and efficient to train.
2. **Baseline Performance**: Establishes a solid baseline of 29.7% accuracy (more than 2x random chance).
3. **Fast Training**: Completes training in approximately 2 hours on standard hardware.
4. **Low Parameters**: The model has fewer parameters compared to more complex architectures, making it memory-efficient.

### Limitations

1. **Limited Temporal Modeling**: The simple GRU layers don't fully capture the complex temporal relationships in emotional speech.
2. **Feature Extraction Depth**: Shallow convolutional layers may not extract sufficiently discriminative features.
3. **Context Awareness**: The model lacks attention mechanisms to focus on the most emotionally salient parts of speech.
4. **Confusion Among Similar Emotions**: High confusion rates between similar emotion pairs (e.g., neutral/calm).



## 6. Key Learnings and Next Steps

From the Base Model implementation, we gained several insights that informed the development of subsequent models:

1. **Need for Better Feature Extraction**: The Enhanced Model should incorporate deeper convolutional layers.
2. **Importance of Attention**: Adding attention mechanisms could help focus on emotionally relevant audio segments.
3. **Temporal Modeling Improvement**: More sophisticated recurrent structures could better capture emotion dynamics.
4. **Data Augmentation**: Techniques like time stretching and pitch shifting might improve generalization.

In the next notebook, we'll explore the Enhanced Model (31.5% accuracy), which addresses some of these limitations through attention mechanisms and improved feature extraction. 